# Formação de Docente no Brasil

## 1 - Importação dos pacotes

In [ ]:
# Imports
# Manipulação de dados e números
import pandas as pd
import numpy as np

# Base de dados
import basedosdados as bd

# Gráficos
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Manipulação de data
import datetime

# Rejeição de mensagens 'warnings'
import warnings
warnings.filterwarnings('ignore')

## 2 - Extração e pré-processamento

In [ ]:
# Criação da query para carregamento dos dados, redução e carregamento dos dados
query = '''
SELECT uf.*, dicionario.valor
FROM `basedosdados.br_inep_formacao_docente.uf` AS uf
JOIN `basedosdados.br_inep_formacao_docente.dicionario` AS dicionario
ON CAST(uf.grupo AS STRING) = dicionario.chave
WHERE dicionario.id_tabela = "regiao"
'''
df = bd.read_sql(query,
                billing_project_id='analise1-rgs')

## 3 - Análise Exploratória

In [ ]:
df.columns = ['ano','regiao','sigla_uf','tipo_localizacao','rede','modalidade','grupo','percentual','categoria_docente']
df.head()

In [ ]:
# Itens distintos em cada variável
df.nunique()

In [ ]:
# Itens nulos nas variáveis
df.isnull().sum()

In [ ]:
# Remoção de valores nulos da base de dados
df = df.dropna()

In [ ]:
df['ano'] = pd.to_datetime(df['ano'], format='%Y').dt.year
df['ano'].dtype

## 4 - Transformação

In [ ]:
# Selecionando os dados do ano de 2020
df2020 = df[df['ano']==2020]

In [ ]:
df2020['rede'].unique()

In [ ]:
# Separando a modalidade categoria_docente e percentual
df2020perc = df[['rede','modalidade','percentual','categoria_docente']]

In [ ]:
# Remoção de algumas observações que não entrarão na análise
df2020perc = df2020perc.loc[~df2020perc['modalidade'].isin(['ensino fundamental anos finais', 'ensino fundamental anos iniciais'])]
df2020perc = df2020perc.loc[df2020perc['rede'].isin(['total'])]
df2020perc = df2020perc[['modalidade','percentual','categoria_docente']]

In [ ]:
# Agrupamento de modalidade, categoria_docente e média do percentual.
df2020perc = df2020perc.groupby(['modalidade','categoria_docente'])['percentual'].mean().reset_index().round(2)

In [ ]:
# Segunda transformação
ano_catdoc = df.loc[df['rede'].isin(['total'])]
ano_catdoc = ano_catdoc[['ano','categoria_docente','percentual']]
ano_catdoc = ano_catdoc.groupby(['ano','categoria_docente'])['percentual'].mean().reset_index()

In [ ]:
# Terceira transformação
docente_uf = df2020[['sigla_uf','categoria_docente','percentual']]
docente_uf = docente_uf.groupby(['sigla_uf','categoria_docente'])['percentual'].mean().reset_index().round(2).sort_index()

In [ ]:
# Quarta transformação
map_redes = df2020.loc[~df2020['rede'].isin(['total'])]
map_redes = map_redes[['rede','grupo','categoria_docente','percentual']]
map_redes = map_redes.groupby(['rede','grupo','categoria_docente'])['percentual'].mean().reset_index().round(2)

map_redes_est = map_redes.loc[map_redes['rede'].isin(['estadual'])]
map_redes_fed = map_redes.loc[map_redes['rede'].isin(['federal'])]
map_redes_mun = map_redes.loc[map_redes['rede'].isin(['municipal'])]
map_redes_priv = map_redes.loc[map_redes['rede'].isin(['privada'])]
map_redes_pub = map_redes.loc[map_redes['rede'].isin(['pública'])]

## 5 - Criação de Gráficos e Visualização

In [ ]:
# Primeiro gráfico (Barras empilhadas)
# Criando objeto fig com a função go.Figure()
fig = go.Figure(

# Definindo as cores das barras referente a qualificação dos professores
color_mapping = {
    'Docentes com formação superior com complementação pedagógica em área diferente da qual leciona': '#aaaaaa',
    'Docentes com formação superior de bacharelado sem complementação pedagógica na mesma área da disciplina que leciona': '#02153d',
    'Docentes com formação superior de licenciatura com complementação pedagógica na mesma área da disciplina que leciona': '#4b227a',
    'Docentes com formação superior não considerada nas categorias anteriores': '#0197af',
    'Docentes sem formação superior': '#00eed0'
}

# Escolhendo novos nomes dos valores do eixo x
novo_rotulo = ['Educação Infantil', 'EJA EF', 'EJA EM', 'Ensino Fundamental', 'Ensino Médio']

# Criando o BarPlot
fig.add_trace(go.Bar(
        x=df2020perc['modalidade'],
        y=df2020perc['percentual'],
        orientation='v',
        marker=dict(
        color=df2020perc['categoria_docente'].map(color_mapping)
        ),
        hovertext=df2020perc['categoria_docente']
))

# Alterando a estética
fig.update_layout(
    legend_title='Categorias de Docentes',
    title='Percentual da qualificação dos professores nas diferentes categorias de ensino do Brasil',
    title_font=dict(
            {'family':'Segoe UI',
            'size':18}),
        annotations=[
        dict(
            text="No ano de 2020",
            showarrow=False,
            xref='paper',
            yref='paper',
            x=-0.041,
            y=1.12,
            font=dict(
                {'family':'Segoe UI',
                'size':14}
            )
        )
    ],
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(df2020perc['modalidade']))),
        ticktext=novo_rotulo),
        font=dict(
            {'family':'Segoe UI',
            'size':12}),
     plot_bgcolor='rgb(244, 244, 244)'
    yaxis_title='Percentual (%)')
    


In [ ]:
# Criando o gráfico de linhas
fig1 = px.line(
        ano_catdoc,
        x='ano',
        y='percentual',
        color='categoria_docente',
        width=500,
        color_discrete_map=color_mapping
)

fig1.update_traces(line=dict(width=4))

fig1.update_layout(
    title='Percentual de títulos de professores nas escolas do Brasil entre 2013 a 2020',
    title_font=dict(
            {'family':'Segoe UI',
            'size':18}),
    plot_bgcolor='rgb(249, 249, 249)',
    legend=dict(
        xanchor='center',
        yanchor='top',
        y=-0.2,
        x=0.5,
        title='Título do docente',
        title_font=dict(
            {'family':'Segoe UI',
            'size':16}),
        font=dict(
            {'family':'Segoe UI',
            'size':10}),
    ),
    xaxis=dict(
        title='Ano',
        title_font=dict(
            {'family':'Segoe UI',
            'size':16})),
    yaxis=dict(
        title='Percentual (%)',
        title_font=dict(
            {'family':'Segoe UI',
            'size':16}))
    

)

In [ ]:
fig2 = px.bar(
        docente_uf,
        x='sigla_uf',
        y='percentual',
        color='categoria_docente',
        barmode='stack',
        color_discrete_map=color_mapping
)

fig2.update_layout(
    title='Porcentagem de tipo de título de docente por UF (2020)',
    font=dict(
        {'family':'Segoe UI'}),
    xaxis=dict(
        categoryorder='total descending',
        title='Estado',
        title_font=dict(
            {'family':'Segoe UI',
            'size':16})),
    legend=dict(
        xanchor='center',
        yanchor='top',
        y=-0.2,
        x=0.5,
        title='Título do docente',
        title_font=dict(
            {'family':'Segoe UI',
            'size':16}),
        font=dict(
            {'family':'Segoe UI',
            'size':10})),
    yaxis=dict(
        title='Percentual (%)',
        title_font=dict(
            {'family':'Segoe UI',
            'size':16})))

In [ ]:
# Definindo as cores dos pie-charts
col_mapping = ['rgb(75,34,122)', 'rgb(2,21,61)', 'rgb(170,170,170)', 'rgb(1,151,175)', 'rgb(0,238,208)']


# Definindo o número de linhas e colunas
specs = [[{'type':'domain'}, {}, {'type':'domain'},{}, {'type':'domain'}],
         [{},{'type':'domain'},{},{'type':'domain'},{}]]
fig3 = make_subplots(rows=2, cols=5, specs=specs,
                   subplot_titles=['Estadual','','Federal','','Municipal','','Privada','','Pública',''],)

# Definindo pie-charts
fig3.add_trace(go.Pie(labels=map_redes_est['categoria_docente'], values=map_redes_est['percentual'],
                      name='Estadual', marker_colors=col_mapping), row=1, col=1)
fig3.add_trace(go.Pie(labels=map_redes_fed['categoria_docente'], values=map_redes_fed['percentual'],
                      name='Federal', marker_colors=col_mapping), row=1, col=3)
fig3.add_trace(go.Pie(labels=map_redes_mun['categoria_docente'], values=map_redes_mun['percentual'],
                      name='Municipal', marker_colors=col_mapping), row=1, col=5)
fig3.add_trace(go.Pie(labels=map_redes_priv['categoria_docente'], values=map_redes_priv['percentual'],
                      name='Privada', marker_colors=col_mapping), row=2, col=2)
fig3.add_trace(go.Pie(labels=map_redes_pub['categoria_docente'], values=map_redes_pub['percentual'],
                      name='Pública', marker_colors=col_mapping), row=2, col=4)

# Ajuste do layout do pie chart
fig3.update_traces(hoverinfo='label+percent+name')
fig3.update_layout(title={
        'text': "Formação dos professores das diferentes redes de ensino",
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'family': 'Segoe UI', 'size': 20}},
                  showlegend=False)

# Ajustando os subtítulos
fig3.update_annotations(
    font=dict(family="Segoe UI", size=16),
    yshift=15)  # Ajuste a altura dos subtítulos aqui


fig3 = go.Figure(fig)
fig3.show()


## 6 - Exportação e Salvamento

In [ ]:
# Save da base de dados da primeira análise em arquivo .csv
df2020perc.to_csv('df2020perc.csv', index=False, encoding='latin-1')

ano_catdoc.to_csv('ano_catdoc.csv', index=False, encoding='latin-1')

docente_uf.to_csv('docente_uf.csv', index=False, encoding='latin-1')

map_redes_est.to_csv('map_redes_est.csv', index=False, encoding='latin-1')
map_redes_fed.to_csv('map_redes_fed.csv', index=False, encoding='latin-1')
map_redes_mun.to_csv('map_redes_mun.csv', index=False, encoding='latin-1')
map_redes_priv.to_csv('map_redes_priv.csv', index=False, encoding='latin-1')
map_redes_pub.to_csv('map_redes_pub.csv', index=False, encoding='latin-1')